In [3]:
'''This script identifies 1000 48nt fragments from
1000 different genes of S. cerevisiae.
Every 4th gene in descending order of expression is picked.
The fragment is the region from 253nt to the 300nt of each ORF.
'''

import pandas as pd
import pyfaidx
from HTSeq import GFF_Reader
import re
import urllib
from Bio.SeqUtils.CodonUsage import CodonAdaptationIndex
import os

In [ ]:
if not os.path.exists("../tables"):
    os.makedirs("../tables")

In [4]:
orf_seq_file = '../db/orf_coding_all_R64-1-1_20110203.fasta'
annotations = '../db/saccharomyces_cerevisiae_R64-1-1_20110208.gff'

In [5]:
# read in the orf sequences as a python dictionary--------------------
orfseqs = pyfaidx.Fasta(orf_seq_file)

# read rna-seq and ribosome profiling data from Weinberg 2016---------
weinberg2016data = pd.read_table('../weinberg2016/GSE53313_Cerevisiae_RNA_RPF.txt',
    index_col=0, skiprows=3, names=['id', 'mrna', 'rpf', 'te'],
    dtype={'id': str, 'mrna': float, 'rpf': float, 'te': float}
)

In [ ]:
# read gene names and functions into a dataframe----------------------
handle = GFF_Reader(annotations)
genes = dict()
try:
    for rec in handle:
        if rec.type != 'gene':
            continue
        if not re.search('Verified', rec.attr['orf_classification']):
            continue
        note = urllib.unquote(urllib.unquote(rec.attr['Note']))
        if 'gene' in rec.attr:
            gene = rec.attr['gene']
        else:
            gene = rec.name
        genes[rec.name] = {
            'note': note,
            'gene': gene,
        }
except ValueError:
    pass
genes = pd.DataFrame.from_dict(genes, orient='index')

In [ ]:
# combine annotations with expression data----------------------------
weinberg2016data = weinberg2016data.join(
    genes, how='inner').sort_values(
        by='rpf', ascending=False)

In [ ]:
# generate a codon adaptation index from all verified genes-----------
genesforcai = open('../tables/genesforcai.fasta', 'w')
for gene in weinberg2016data.index:
    name = weinberg2016data.ix[gene]['gene']
    note = weinberg2016data.ix[gene]['note']
    genesforcai.write('>{0}:{1}\n'.format(gene, name))
    for line in orfseqs[gene]:
        genesforcai.write(str(line) + '\n')
genesforcai.close()
# use CAI module from biopython to get cai
yeastcai = CodonAdaptationIndex()
yeastcai.generate_index('../tables/genesforcai.fasta')

In [ ]:
# write a 48nt fragment for primer design-----------------------------
yeastseqs = open('../tables/yeastorffrags.csv', 'w')
genecount = 0
caiforfrags = dict()
for gene in weinberg2016data.index[::2]:
    if len(orfseqs[gene]) < 300:
        continue
    frag = str(orfseqs[gene][252:300])
    # check that there are no stop codons
    for pos in range(0, len(frag), 3):
        codon = frag[pos:pos+3]
        if codon in ['TAA', 'TAG', 'TGA']:
            raise 'Stop codon in fragment. Should not be here.'
    # track the cai for this fragment
    caiforfrags[gene] = yeastcai.cai_for_gene(frag)
    yeastseqs.write(frag + '\n')
    genecount += 1
    if genecount >= 1904:
        break
yeastseqs.close()

In [ ]:
# print the name of gene and corresponding cai for fragment-----------
print 'Top 10 genes with highest CAI for the cloned fragment:'
print weinberg2016data.join(
    pd.Series(caiforfrags, name='caiforfrag'),
    how='inner').sort_values(
        by='caiforfrag',
        ascending=False)[['gene', 'caiforfrag']][:10]

# clean up files -----------------------------------------------------
os.remove('../tables/genesforcai.fasta')